In [1]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from keras import layers, Sequential
from keras.models import Model, model_from_json
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
X_train = np.load('train_recurrence_plot_rescaled_v1.npy')
Y_train = np.load('Y_train_diffused_0.5_TH.npy')

In [206]:
version = 3

In [3]:
Y_train.shape

(390, 13)

In [4]:
X_train.shape

(390, 256, 256, 2)

In [3]:
def build_CNN():
    model = Sequential()
#     model.add(layers.InputLayer(input_shape=(512,512,2), name='Input_Layer'))    
    model.add(layers.Conv2D(filters=32, kernel_size=5, padding='valid', input_shape=(256,256,2), name='Input_Layer'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D())
#     model.add(layers.Dropout(0.1))
  
    
#     model.add(layers.Conv2D(filters=32, kernel_size=3, padding='valid'))
#     model.add(layers.BatchNormalization())
#     model.add(layers.Activation('relu'))
#     model.add(layers.MaxPooling2D())
# #     model.add(layers.Dropout(0.1))

    model.add(layers.Conv2D(filters=64, kernel_size=5, padding='valid'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D())
#     model.add(layers.Dropout(0.1))
    

    model.add(layers.Conv2D(filters=128, kernel_size=5, padding='valid'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D())
#     model.add(layers.Dropout(0.1))
    
    model.add(layers.Flatten())
#     model.add(layers.Dropout(0.4))
    model.add(layers.Dense(128))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(13,activation='softmax',name = 'Output_Layer'))
    opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [107]:
train_x, val_x, train_y, val_y = train_test_split(X_train,Y_train, test_size=0.2, random_state=7)
train_x.shape

(312, 256, 256, 2)

In [4]:
from utils import plot_training_summary
from utils import TimeSummary
from utils import save_keras_dataset_to_disk
from keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, min_delta=0.5, patience=50, baseline=0.9)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True, verbose=1)
from keras.callbacks import TensorBoard
import time
NAME = 'CNN_RP {}'.format(int(time.time()))
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

# Cross-Validation Block

In [222]:
Y_train_one_col = np.zeros((390,1))
for i in np.arange(13):
    Y_train_one_col[i*30:i*30+30,0] = int(i)
Y_train_one_col.astype(int);

In [223]:
cvscores = []
# model = build_CNN()
# model.summary()
Kfold_Stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=7)
for j, (train_idx, val_idx) in enumerate(Kfold_Stratified.split(X_train,Y_train_one_col)):
    print('\nFold:', j+1)
    X_train_cv = X_train[train_idx]
    y_train_cv = Y_train[train_idx]
    X_valid_cv = X_train[val_idx]
    y_valid_cv= Y_train[val_idx]
    
    model_cv = build_CNN()
    model_cv.fit(X_train_cv,
                  y_train_cv,
                  epochs = 1000,
                  validation_data = (X_valid_cv,y_valid_cv)
                 )
    scores = model_cv.evaluate(X_valid_cv, y_valid_cv, verbose = 0)
    cvscores.append(scores[1]*100)
print(np.mean(cvscores), np.std(cvscores))


Fold: 1
Train on 312 samples, validate on 78 samples
Epoch 1/1000
312/312 [==============================] - 5s 16ms/step - loss: 2.1227 - acc: 0.3045 - val_loss: 1.4636 - val_acc: 0.5769
Epoch 2/1000
312/312 [==============================] - 1s 4ms/step - loss: 1.6033 - acc: 0.3942 - val_loss: 1.1220 - val_acc: 0.6795
Epoch 3/1000
312/312 [==============================] - 1s 4ms/step - loss: 1.4036 - acc: 0.4615 - val_loss: 1.0304 - val_acc: 0.7051
Epoch 4/1000
312/312 [==============================] - 1s 4ms/step - loss: 1.6863 - acc: 0.4199 - val_loss: 1.0704 - val_acc: 0.7051
Epoch 5/1000
312/312 [==============================] - 1s 4ms/step - loss: 1.4457 - acc: 0.4487 - val_loss: 1.0973 - val_acc: 0.5897
Epoch 6/1000
312/312 [==============================] - 1s 4ms/step - loss: 1.2355 - acc: 0.5513 - val_loss: 0.9539 - val_acc: 0.9231
Epoch 7/1000
312/312 [==============================] - 1s 4ms/step - loss: 1.2739 - acc: 0.5192 - val_loss: 0.9375 - val_acc: 0.8333
Epoch 8

312/312 [==============================] - 1s 4ms/step - loss: 0.4638 - acc: 0.9904 - val_loss: 0.5028 - val_acc: 0.9744
Epoch 122/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4890 - acc: 0.9712 - val_loss: 0.4744 - val_acc: 0.9615
Epoch 123/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.5040 - acc: 0.9647 - val_loss: 0.4790 - val_acc: 0.9872
Epoch 124/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4757 - acc: 0.9840 - val_loss: 0.5222 - val_acc: 0.9872
Epoch 125/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4885 - acc: 0.9647 - val_loss: 0.5440 - val_acc: 0.9615
Epoch 126/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4729 - acc: 0.9840 - val_loss: 0.4656 - val_acc: 0.9872
Epoch 127/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4725 - acc: 0.9840 - val_loss: 0.5085 - val_acc: 0.9615
Epoch 128/1000
312/312 [==============================] - 1s 4m

312/312 [==============================] - 1s 4ms/step - loss: 0.4522 - acc: 0.9872 - val_loss: 0.4531 - val_acc: 0.9872
Epoch 242/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4465 - acc: 0.9872 - val_loss: 0.4373 - val_acc: 0.9744
Epoch 243/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4378 - acc: 0.9840 - val_loss: 0.4769 - val_acc: 0.9359
Epoch 244/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4507 - acc: 0.9872 - val_loss: 0.4419 - val_acc: 0.9872
Epoch 245/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4315 - acc: 0.9936 - val_loss: 0.4620 - val_acc: 0.9872
Epoch 246/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4434 - acc: 0.9872 - val_loss: 0.4904 - val_acc: 0.9872
Epoch 247/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4397 - acc: 0.9904 - val_loss: 0.5006 - val_acc: 0.9615
Epoch 248/1000
312/312 [==============================] - 1s 4m

312/312 [==============================] - 1s 4ms/step - loss: 0.4306 - acc: 0.9936 - val_loss: 0.4410 - val_acc: 0.9744
Epoch 362/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4209 - acc: 0.9904 - val_loss: 0.4352 - val_acc: 0.9872
Epoch 363/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4225 - acc: 0.9936 - val_loss: 0.4484 - val_acc: 0.9872
Epoch 364/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4211 - acc: 0.9936 - val_loss: 0.4421 - val_acc: 0.9744
Epoch 365/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4255 - acc: 0.9936 - val_loss: 0.4471 - val_acc: 0.9872
Epoch 366/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4216 - acc: 0.9936 - val_loss: 0.4699 - val_acc: 0.9744
Epoch 367/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4200 - acc: 0.9904 - val_loss: 0.4278 - val_acc: 0.9744
Epoch 368/1000
312/312 [==============================] - 1s 4m

312/312 [==============================] - 1s 4ms/step - loss: 0.4214 - acc: 0.9904 - val_loss: 0.4391 - val_acc: 0.9744
Epoch 482/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4169 - acc: 0.9904 - val_loss: 0.4424 - val_acc: 0.9744
Epoch 483/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4157 - acc: 0.9936 - val_loss: 0.4394 - val_acc: 0.9744
Epoch 484/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4175 - acc: 0.9936 - val_loss: 0.4748 - val_acc: 0.9744
Epoch 485/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4196 - acc: 0.9904 - val_loss: 0.4884 - val_acc: 0.9744
Epoch 486/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4138 - acc: 0.9936 - val_loss: 0.4186 - val_acc: 0.9744
Epoch 487/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4153 - acc: 0.9968 - val_loss: 0.4382 - val_acc: 0.9744
Epoch 488/1000
312/312 [==============================] - 1s 4m

...

312/312 [==============================] - 1s 4ms/step - loss: 0.4079 - acc: 0.9968 - val_loss: 0.4591 - val_acc: 0.9744
Epoch 446/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4095 - acc: 0.9936 - val_loss: 0.4614 - val_acc: 0.9744
Epoch 447/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4058 - acc: 0.9968 - val_loss: 0.4473 - val_acc: 0.9744
Epoch 448/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4052 - acc: 0.9968 - val_loss: 0.4888 - val_acc: 0.9744
Epoch 449/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4194 - acc: 0.9872 - val_loss: 0.4521 - val_acc: 0.9744
Epoch 450/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4136 - acc: 0.9904 - val_loss: 0.4606 - val_acc: 0.9744
Epoch 451/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4101 - acc: 0.9968 - val_loss: 0.4533 - val_acc: 0.9744
Epoch 452/1000
312/312 [==============================] - 1s 4m

312/312 [==============================] - 1s 4ms/step - loss: 0.4009 - acc: 0.9936 - val_loss: 0.4863 - val_acc: 0.9615
Epoch 566/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3995 - acc: 1.0000 - val_loss: 0.4650 - val_acc: 0.9744
Epoch 567/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4097 - acc: 0.9904 - val_loss: 0.4775 - val_acc: 0.9744
Epoch 568/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4035 - acc: 0.9968 - val_loss: 0.4635 - val_acc: 0.9744
Epoch 569/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3999 - acc: 0.9936 - val_loss: 0.5022 - val_acc: 0.9615
Epoch 570/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4007 - acc: 0.9936 - val_loss: 0.4784 - val_acc: 0.9744
Epoch 571/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4017 - acc: 0.9904 - val_loss: 0.4562 - val_acc: 0.9744
Epoch 572/1000
312/312 [==============================] - 1s 4m

312/312 [==============================] - 1s 4ms/step - loss: 0.4030 - acc: 0.9872 - val_loss: 0.4461 - val_acc: 0.9744
Epoch 686/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3991 - acc: 0.9936 - val_loss: 0.4396 - val_acc: 0.9744
Epoch 687/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4014 - acc: 0.9968 - val_loss: 0.4428 - val_acc: 0.9744
Epoch 688/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3985 - acc: 0.9968 - val_loss: 0.4665 - val_acc: 0.9615
Epoch 689/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3986 - acc: 0.9936 - val_loss: 0.4583 - val_acc: 0.9744
Epoch 690/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4025 - acc: 0.9968 - val_loss: 0.4467 - val_acc: 0.9744
Epoch 691/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4034 - acc: 0.9872 - val_loss: 0.4360 - val_acc: 0.9744
Epoch 692/1000
312/312 [==============================] - 1s 4m

312/312 [==============================] - 1s 4ms/step - loss: 0.3971 - acc: 0.9968 - val_loss: 0.4600 - val_acc: 0.9744
Epoch 806/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3976 - acc: 0.9936 - val_loss: 0.4827 - val_acc: 0.9744
Epoch 807/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3933 - acc: 1.0000 - val_loss: 0.4685 - val_acc: 0.9744
Epoch 808/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4019 - acc: 0.9936 - val_loss: 0.4754 - val_acc: 0.9744
Epoch 809/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.4016 - acc: 0.9936 - val_loss: 0.4724 - val_acc: 0.9744
Epoch 810/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3955 - acc: 1.0000 - val_loss: 0.4541 - val_acc: 0.9744
Epoch 811/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3909 - acc: 1.0000 - val_loss: 0.4740 - val_acc: 0.9744
Epoch 812/1000
312/312 [==============================] - 1s 4m

312/312 [==============================] - 1s 4ms/step - loss: 0.3965 - acc: 0.9936 - val_loss: 0.4735 - val_acc: 0.9744
Epoch 926/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3976 - acc: 0.9968 - val_loss: 0.4665 - val_acc: 0.9744
Epoch 927/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3970 - acc: 0.9936 - val_loss: 0.4513 - val_acc: 0.9744
Epoch 928/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3920 - acc: 0.9968 - val_loss: 0.4352 - val_acc: 0.9744
Epoch 929/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3915 - acc: 0.9936 - val_loss: 0.4538 - val_acc: 0.9744
Epoch 930/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3964 - acc: 1.0000 - val_loss: 0.4431 - val_acc: 0.9744
Epoch 931/1000
312/312 [==============================] - 1s 4ms/step - loss: 0.3913 - acc: 0.9936 - val_loss: 0.4416 - val_acc: 0.9744
Epoch 932/1000
312/312 [==============================] - 1s 4m

In [224]:
cvscores

[97.43589743589743, 100.0, 100.0, 98.71794871794873, 97.43589697740018]

In [112]:
# stop here

# Train and Test block

In [113]:
# time_summary = TimeSummary()
# model = build_CNN()
# # model.summary()
# summary = model.fit(train_x,
#                       train_y,
#             #           batch_size = 5,
#                       epochs = 1000,
#                       verbose = 1,
#                       callbacks = [time_summary,tensorboard],
#                       validation_data=(val_x,val_y)       
      
#          )

# Train with all data (no validation block)

In [ ]:
# time_summary = TimeSummary()
# model = build_CNN()
# # model.summary()
summary = model.fit(X_train,
                      Y_train,
            #           batch_size = 5,
                      epochs = 5,
                      verbose = 1,
                      callbacks = [time_summary,tensorboard]
#                       validation_data=(val_x,val_y)       
      
         )

In [ ]:
plot_training_summary(summary, time_summary)

In [209]:
version

3

In [210]:
# model.save_weights('model_weight_800epoch.h5')
# print('saved model weights')
model_json = model.to_json()
with open(f"model_v{version}.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(f"model_weight_v{version}.h5")
print("Saved model to disk")

Saved model to disk


In [5]:
version = '_Final'

In [6]:
json_file = open(f'model_v{version}.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(f"model_weight_v{version}.h5")
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

loaded_model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print("Loaded model from disk")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model from disk


In [9]:
%%timeit -r 100 
result = loaded_model.predict(test_set_4.reshape(1,256,256,2))

4.54 ms ± 249 µs per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [7]:
test_160_recurrence = np.load('test_160_recurrence_256Deg.npy')
test_230_recurrence = np.load('test_230_recurrence_256Deg.npy')
test_315_recurrence = np.load('test_315_recurrence_256Deg.npy')
test_405_recurrence = np.load('test_405_recurrence_256Deg.npy')

In [5]:
test_160_recurrence.shape

(30, 256, 256, 2)

In [8]:
test_set_4 = test_405_recurrence[20,:,:,:]

In [14]:
test_set_4.shape

(256, 256, 2)

In [15]:
np.save('test_set_4',test_set_4)

In [12]:
result = loaded_model.predict(test_set_4.reshape(1,256,256,2))

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node Input_Layer_1/convolution}}]]
	 [[{{node Output_Layer_1/Softmax}}]]

In [13]:
result

NameError: name 'result' is not defined

In [213]:
test_160_result = loaded_model.predict(test_160_recurrence)
for i in np.arange(20):
    print(['90_Deg','120_Deg','150_Deg','180_Deg','210_Deg','240_Deg',
             '270_Deg','300_Deg','330_Deg','360_Deg','390_Deg','420_Deg','450_Deg',
             'Test_160_Deg','Test_230_Deg','Test_315_Deg','Test_405_Deg'
            ][np.argmax(test_160_result[i,:])], 'with probability of:', test_160_result[i,np.argmax(test_160_result[i,:])])

90_Deg with probability of: 0.85025257
180_Deg with probability of: 0.5774015
180_Deg with probability of: 0.5715629
180_Deg with probability of: 0.54681176
180_Deg with probability of: 0.54940265
180_Deg with probability of: 0.5677043
180_Deg with probability of: 0.5536864
180_Deg with probability of: 0.46915427
150_Deg with probability of: 0.46676654
150_Deg with probability of: 0.5122801
150_Deg with probability of: 0.51254386
180_Deg with probability of: 0.5194991
180_Deg with probability of: 0.53364104
180_Deg with probability of: 0.54950064
180_Deg with probability of: 0.53412586
180_Deg with probability of: 0.5962488
180_Deg with probability of: 0.5374931
180_Deg with probability of: 0.525327
180_Deg with probability of: 0.5517977
180_Deg with probability of: 0.49375987


In [214]:
for i in np.arange(30):
    print(test_160_result[i,2],test_160_result[i,3])

0.016079128 0.0007160953
0.35805845 0.5774015
0.36120167 0.5715629
0.3746017 0.54681176
0.37926334 0.54940265
0.3719975 0.5677043
0.38359162 0.5536864
0.46124873 0.46915427
0.46676654 0.46633264
0.5122801 0.42094648
0.51254386 0.42606843
0.41202846 0.5194991
0.39378658 0.53364104
0.37812233 0.54950064
0.40135553 0.53412586
0.3580015 0.5962488
0.41106048 0.5374931
0.42112663 0.525327
0.39252692 0.5517977
0.45525417 0.49375987
0.48444974 0.4587367
0.5243657 0.42371812
0.41615745 0.54478794
0.4085271 0.55264837
0.42252895 0.5261007
0.3641809 0.58856195
0.46814367 0.4892645
0.37614292 0.5602246
0.36605915 0.58898425
0.34402105 0.6161914


In [215]:
test_230_result = loaded_model.predict(test_230_recurrence)
for i in np.arange(20):
    print(['90_Deg','120_Deg','150_Deg','180_Deg','210_Deg','240_Deg',
             '270_Deg','300_Deg','330_Deg','360_Deg','390_Deg','420_Deg','450_Deg',
             'Test_160_Deg','Test_230_Deg','Test_315_Deg','Test_405_Deg'
            ][np.argmax(test_230_result[i,:])], 'with probability of:', test_230_result[i,np.argmax(test_230_result[i,:])])

90_Deg with probability of: 0.842989
240_Deg with probability of: 0.57314503
240_Deg with probability of: 0.66043144
240_Deg with probability of: 0.6295307
240_Deg with probability of: 0.5966058
240_Deg with probability of: 0.6315352
240_Deg with probability of: 0.5893155
210_Deg with probability of: 0.53899884
210_Deg with probability of: 0.5022856
240_Deg with probability of: 0.53773826
240_Deg with probability of: 0.54953843
240_Deg with probability of: 0.5952632
240_Deg with probability of: 0.64955443
240_Deg with probability of: 0.610035
240_Deg with probability of: 0.562356
240_Deg with probability of: 0.5866006
240_Deg with probability of: 0.5508235
240_Deg with probability of: 0.52757347
240_Deg with probability of: 0.5426571
240_Deg with probability of: 0.5896349


In [216]:
for i in np.arange(30):
    print(test_230_result[i,4],test_230_result[i,5])

0.0016716678 0.01113257
0.41943783 0.57314503
0.33133718 0.66043144
0.3621861 0.6295307
0.3942265 0.5966058
0.36018792 0.6315352
0.40150803 0.5893155
0.53899884 0.44836432
0.5022856 0.4888522
0.45567802 0.53773826
0.44482675 0.54953843
0.39480895 0.5952632
0.31822097 0.64955443
0.362724 0.610035
0.42663303 0.562356
0.40729547 0.5866006
0.44426873 0.5508235
0.46536043 0.52757347
0.4504856 0.5426571
0.40204102 0.5896349
0.41423193 0.580322
0.4596584 0.5342082
0.42694595 0.56550145
0.47162262 0.5215684
0.4617439 0.5325482
0.41432136 0.57965046
0.45730993 0.53541666
0.4324446 0.5607652
0.54525924 0.44096512
0.39505106 0.5947093


In [221]:
test_315_result = loaded_model.predict(test_315_recurrence)
for i in np.arange(30):
    print(['90_Deg','120_Deg','150_Deg','180_Deg','210_Deg','240_Deg',
             '270_Deg','300_Deg','330_Deg','360_Deg','390_Deg','420_Deg','450_Deg',
             'Test_160_Deg','Test_230_Deg','Test_315_Deg','Test_405_Deg'
            ][np.argmax(test_315_result[i,:])], 'with probability of:', test_315_result[i,np.argmax(test_315_result[i,:])])

90_Deg with probability of: 0.80226356
300_Deg with probability of: 0.64066106
300_Deg with probability of: 0.56873655
300_Deg with probability of: 0.66684556
300_Deg with probability of: 0.60890186
300_Deg with probability of: 0.66399974
300_Deg with probability of: 0.6232939
300_Deg with probability of: 0.5577825
300_Deg with probability of: 0.60423464
300_Deg with probability of: 0.6204527
300_Deg with probability of: 0.59962595
300_Deg with probability of: 0.57245946
300_Deg with probability of: 0.6136788
300_Deg with probability of: 0.6057818
300_Deg with probability of: 0.56725895
300_Deg with probability of: 0.6564766
300_Deg with probability of: 0.61243904
300_Deg with probability of: 0.5247359
300_Deg with probability of: 0.5334953
300_Deg with probability of: 0.65535235
300_Deg with probability of: 0.5676271
300_Deg with probability of: 0.6267009
300_Deg with probability of: 0.6203211
300_Deg with probability of: 0.59202445
300_Deg with probability of: 0.52472645
300_Deg with

In [218]:
for i in np.arange(30):
    print(test_315_result[i,7],test_315_result[i,8])

0.01947382 0.00298361
0.64066106 0.31979775
0.56873655 0.39058957
0.66684556 0.2788439
0.60890186 0.3479695
0.66399974 0.31463152
0.6232939 0.34169775
0.5577825 0.42003086
0.60423464 0.3414476
0.6204527 0.2765649
0.59962595 0.37885168
0.57245946 0.39910254
0.6136788 0.35621095
0.6057818 0.35781795
0.56725895 0.36927524
0.6564766 0.31846064
0.61243904 0.34920257
0.5247359 0.4569486
0.5334953 0.44582215
0.65535235 0.28083172
0.5676271 0.40411946
0.6267009 0.33516642
0.6203211 0.35938048
0.59202445 0.38207972
0.52472645 0.4431042
0.55130404 0.41347528
0.5072635 0.46222687
0.57127184 0.40090874
0.6774248 0.24886864
0.6438867 0.2785172


In [219]:
test_405_result = loaded_model.predict(test_405_recurrence)
for i in np.arange(30):
    print(['90_Deg','120_Deg','150_Deg','180_Deg','210_Deg','240_Deg',
             '270_Deg','300_Deg','330_Deg','360_Deg','390_Deg','420_Deg','450_Deg',
             'Test_160_Deg','Test_230_Deg','Test_315_Deg','Test_405_Deg'
            ][np.argmax(test_405_result[i,:])], 'with probability of:', test_405_result[i,np.argmax(test_405_result[i,:])])

450_Deg with probability of: 0.53483725
390_Deg with probability of: 0.48869842
420_Deg with probability of: 0.49930608
390_Deg with probability of: 0.48312998
390_Deg with probability of: 0.4360355
390_Deg with probability of: 0.58415174
420_Deg with probability of: 0.51806855
390_Deg with probability of: 0.492926
390_Deg with probability of: 0.54439455
390_Deg with probability of: 0.54185486
390_Deg with probability of: 0.5845041
390_Deg with probability of: 0.6087661
390_Deg with probability of: 0.5118286
390_Deg with probability of: 0.50615823
390_Deg with probability of: 0.5357452
390_Deg with probability of: 0.47636437
390_Deg with probability of: 0.57858396
390_Deg with probability of: 0.5708929
390_Deg with probability of: 0.50699776
390_Deg with probability of: 0.5381469
390_Deg with probability of: 0.6393801
390_Deg with probability of: 0.6456915
390_Deg with probability of: 0.6527564
390_Deg with probability of: 0.52808607
390_Deg with probability of: 0.5942726
390_Deg with 

In [220]:
for i in np.arange(30):
    print(test_405_result[i,10],test_405_result[i,11])

0.054013968 0.019806113
0.48869842 0.45053443
0.4388481 0.49930608
0.48312998 0.4239074
0.4360355 0.39909282
0.58415174 0.3324429
0.4216964 0.51806855
0.492926 0.42931667
0.54439455 0.36278704
0.54185486 0.3785916
0.5845041 0.32014525
0.6087661 0.31286523
0.5118286 0.3885154
0.50615823 0.40969035
0.5357452 0.37446618
0.47636437 0.45878002
0.57858396 0.33042678
0.5708929 0.31088752
0.50699776 0.4102788
0.5381469 0.38926062
0.6393801 0.2733967
0.6456915 0.26959226
0.6527564 0.2622282
0.52808607 0.38495812
0.5942726 0.30342865
0.5152743 0.38666078
0.4059391 0.52790385
0.4534384 0.47229823
0.44467914 0.4758112
0.4245047 0.5112308
